## Исследование надёжности заёмщиков

Учебный проект Яндекс.Практикума. 

Цель: определение влияния различных параметров заемщика на факт возврата им кредита в срок. Результаты исследования будут учтены при построении модели кредитного скоринга.

Источник данных: Яндекс.Практикум

Данные: 21500 записей о заемщиках с указанием базовой информации и сведениях о факте наличия задолженностей.

### Шаг 1. Откройте файл с данными и изучите общую информацию. 

In [1]:
import pandas as pd
from pymystem3 import Mystem
from collections import Counter


data_raw = pd.read_csv('/datasets/data.csv')
data_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Первые 10 строк исследуемого датасета

In [2]:
data_raw.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


### Вывод

Данные представлены в таблице 21525 х 12. 
Из данных о столбцах можно сделать следующие выводы:
1. В столбцах days_employed и total_income присутствуют пропуски
2. В остальных столбцах нет ни пропусков, ни данных не верного типа
3. В столбце days_employed потребуется изменить тип данных на int64 (при переходе к целым числам слабо изменятся 
    значения, а значит слабо изменятся и соответствующие статистические характеристики)
4. В столбце 'days_employed' присутствуют и отрицательные, и положительные значения, помимо этого представленные значения могут отличаться друг от друга на несколько порядков
5. Количество пропусков в days_employed и total_income совпадают, что может указывать на некорректную работу алгоритмов выгрузки данных
6. Может потребоваться внедрение столбцов с id, соответствующих данным из колонок income_type, purpose для оптимизации процесса категоризации данных и исключения связанных с этим процессом ошибок

### Шаг 2. Предобработка данных

### Обработка пропусков

Согласно информации о таблице, количества пропусков в колонке days_employed и total_income совпадают и равны 2174. Необходимо установить, всегда ли наличие пропуска в колонке days_employed означает пропуск в соответствующей строке в колонке total_income. 

In [3]:
na_count = data_raw[data_raw['days_employed'].isna()]['total_income'].isna().count()
na_percent = na_count / 21525
print(f"Количество строк с пропусками одновременно в колонке 'days_employed' и в колонке 'total_income': {na_count}")
print('Процент данных с пропусками: {:.1%}'.format(na_percent))

Количество строк с пропусками одновременно в колонке 'days_employed' и в колонке 'total_income': 2174
Процент данных с пропусками: 10.1%


#### Промежуточный вывод
1. Полученное значение равно 2174. Значит, если в строке отсутствуют данные о рабочем стаже, автоматически отсутствуют и данные о ежемесячном доходе. 
2. Возникновение пропусков является системным, необходимо проверить алгоритмы работы сбора данных из базы данных. 
3. При удалении всех пропусков объем данных сократится на 10.1%, потому пропуски лучше заполнить. Это определенно скажется на одной из целевых метрик, но повысит точность ответов на оставшиеся вопросы.
4. Пропуски будут заполнены следующими значениями: 
  1. Данные в колонке о доходах разбиваются на группы, чтобы выяснить присутствуют ли в колонке выбросы - отрицательные или аномально высокие (в масштабах выборки) значения
  2. Выполняется проверка на адекватность данных в колонке 'gender', различные обозначения пола приводятся к единому виду, данные о неизвестных полах удаляются
  3. Строки датасета без пропусков будут сгруппированы по полу, затем по возрасту: выбор именно этих колонок обусловлен тем, что данные именно в этих колонках наиболее достоверны (группировка, например, по стажу невозможна в силу присутствия большого числа неадекватно завышенных значений - об этом далее). Более того, существуют статистические данные о различии доходов женщин и мужчин, а так же специалистов разного возраста
  4. Для каждой из полученных категорий будут рассчитаны средние значения
  5. Пропуски будут заполнены средним для данного пола и возраста значением

In [4]:
def outlier_search_income(data):
    """
    Функция категоризирует данные по уровню дохода. В случае пропуска значения присваивает группу "Пропуск", которая не
    попадает в выходной датасет
    :data: входной датасет. Исследуемая колонка 'total_income' может содержать пропуски
    :return: сводная таблица: группировка по группе дохода, подсчет количества значений в группе.
             Группы дохода: 'менее 100 тыс.', 'от 100 до 200 тыс.', 'от 200 до 300 тыс.', 'от 300 до 400 тыс.',
             'от 400 до 500 тыс.', 'более 500 тыс.'.
    """

    def total_income_group(row):
        try:
            if row['total_income'] <= 0:
                return 'неположительный доход'
            elif 0 < row['total_income'] <= 100000:
                return 'менее 100 тыс.'
            elif 100000 < row['total_income'] <= 200000:
                return 'от 100 до 200 тыс.'
            elif 200000 < row['total_income'] <= 300000:
                return 'от 200 до 300 тыс.'
            elif 300000 < row['total_income'] <= 400000:
                return 'от 300 до 400 тыс.'
            elif 400000 < row['total_income'] <= 500000:
                return 'от 400 до 500 тыс.'
            elif row['total_income'] > 500000:
                return 'более 500 тыс.'
        except:
            return 'Пропуск'

    data_total_income_group = pd.DataFrame()
    data_total_income_group['total_income'] = data['total_income']
    data_total_income_group['income_group'] = data.apply(total_income_group, axis=1)
    return data_total_income_group.groupby('income_group')['total_income'].count().sort_values(ascending=True)


outlier_search_income(data_raw)

income_group
более 500 тыс.         222
от 400 до 500 тыс.     306
от 300 до 400 тыс.     954
от 200 до 300 тыс.    3584
менее 100 тыс.        4463
от 100 до 200 тыс.    9822
Name: total_income, dtype: int64

Из результатов группировки по доходу следует: 
1. Неположительных значений дохода в данных нет
2. Необходимо проверить данные на наличие аномально высоких (в масштабах выборки) доходов

In [5]:
total_income_max = data_raw['total_income'].max()
print(f'Максимальное значение дохода в выборке: {total_income_max}')

Максимальное значение дохода в выборке: 2265604.028722744


Данные о доходе не содержат выбросов: подсчет средних значений дохода можно выполнять без дополнительной подготовки данных

In [6]:
#проверка значений, представленных в колонке 'gender'
gender_list = data_raw['gender'].unique()
print(gender_list)

['F' 'M' 'XNA']


В данных обнаружены строки с неизвестным полом 'XNA'

In [7]:
# вывод строки с неизестным полом
data_raw.loc[data_raw.loc[:, 'gender'] == 'XNA']

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
10701,0,-2358.600502,24,неоконченное высшее,2,гражданский брак,1,XNA,компаньон,0,203905.157261,покупка недвижимости


Обнаружена единственная строка с неизвестным полом. Перед заполнением пропусков в 'total_income' ее необходимо удалить

In [8]:
# удаление строки с полом 'XNA', сброс индексирования и проверка сводной информации
data_raw = data_raw.drop([10701])
data_raw.reset_index(drop=True)
data_raw.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21524 entries, 0 to 21524
Data columns (total 12 columns):
children            21524 non-null int64
days_employed       19350 non-null float64
dob_years           21524 non-null int64
education           21524 non-null object
education_id        21524 non-null int64
family_status       21524 non-null object
family_status_id    21524 non-null int64
gender              21524 non-null object
income_type         21524 non-null object
debt                21524 non-null int64
total_income        19350 non-null float64
purpose             21524 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.1+ MB


Строка с неизвестным полом была удалена, данные готовы к группировке и заполнению пропусков с использованием данной группировки.

#### Заполнение пропусков в колонке 'total_income'

In [9]:
def replace_na_income_with_mean(df):
    '''
    Функция заполняет пропуски в колонке 'total_income' по следующему механизму: из исходного датасета удаляются все пропуски, 
    оставшиеся данные группируются сначала по полу, затем по возрасту. Для каждой группы вычисляется среднее значение 
    ежемесячного дохода, которое затем проставляется на места пропусков для заёмщиков с НЕНУЛЕВЫМ возрастом
    
    :df_no_na: исходный датасет с полностью удаленными пропусками
    :total_income_mean: результат группировки df_no_na сначала по полу, затем по возрасту, колонка total_income_mean содержит
                        соответствующее данной группе среднее значение месячного дохода
    :df_filled_na: результат слияния total_income_mean и df по колонкам 'gender' и 'dob_years' с сохранением индексирования
                   из df
    :return: df_filled_na, в котором пропуски в колонке 'total_income' для всех заемщиков С НЕНУЛЕВЫМ ВОЗРАСТОМ заполнены 
             средним для данного пола и возраста значением
    '''
    df_no_na = df.dropna()
    total_income_mean = df_no_na.groupby(['gender', 'dob_years']).agg({'total_income': 'mean'}).rename(columns={'total_income': 'total_income_mean'})
    df_filled_na = df.merge(total_income_mean, on=['gender', 'dob_years'], how='left')
    df_filled_na.loc[(df_filled_na.loc[:, 'total_income'].isna()) & (df_filled_na.loc[:, 'dob_years'] != 0), 'total_income'] = df_filled_na.loc[(df_filled_na.loc[:, 'total_income'].isna()) & (df_filled_na.loc[:, 'dob_years'] != 0), 'total_income_mean']
    return df_filled_na


# заполнение пропусков в колонке 'total_income' и вывод первых пяти строк, в которых ранее в 'total_income' стояли пропуски 
data_raw = replace_na_income_with_mean(data_raw)
data_raw.loc[data_raw.loc[:, 'days_employed'].isna()].head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,total_income_mean
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,167269.001754,сыграть свадьбу,167269.001754
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,209297.523153,образование,209297.523153
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,145337.713305,строительство жилой недвижимости,145337.713305
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,152275.289235,сделка с подержанным автомобилем,152275.289235
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,148272.317612,сыграть свадьбу,148272.317612


In [10]:
# вывод информации о текущем содержании таблицы
data_raw.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21524 entries, 0 to 21523
Data columns (total 13 columns):
children             21524 non-null int64
days_employed        19350 non-null float64
dob_years            21524 non-null int64
education            21524 non-null object
education_id         21524 non-null int64
family_status        21524 non-null object
family_status_id     21524 non-null int64
gender               21524 non-null object
income_type          21524 non-null object
debt                 21524 non-null int64
total_income         21514 non-null float64
purpose              21524 non-null object
total_income_mean    21524 non-null float64
dtypes: float64(3), int64(5), object(5)
memory usage: 2.3+ MB


В колонке ежемесячного дохода все еще остаются пропуски.  
Содержание строк с пропусками:

In [11]:
data_raw.loc[data_raw.loc[:, 'total_income'].isna()]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,total_income_mean
1890,0,NaN,0,высшее,0,Не женат / не замужем,4,F,сотрудник,0,NaN,жилье,142763.314708
2284,0,NaN,0,среднее,1,вдовец / вдова,2,F,пенсионер,0,NaN,недвижимость,142763.314708
4064,1,NaN,0,среднее,1,гражданский брак,1,M,компаньон,0,NaN,ремонт жилью,195255.623276
5014,0,NaN,0,среднее,1,женат / замужем,0,F,компаньон,0,NaN,покупка недвижимости,142763.314708
6411,0,NaN,0,высшее,0,гражданский брак,1,F,пенсионер,0,NaN,свадьба,142763.314708
6670,0,NaN,0,Высшее,0,в разводе,3,F,пенсионер,0,NaN,покупка жилой недвижимости,142763.314708
8574,0,NaN,0,среднее,1,женат / замужем,0,F,сотрудник,0,NaN,недвижимость,142763.314708
12402,3,NaN,0,среднее,1,женат / замужем,0,M,сотрудник,0,NaN,операции с коммерческой недвижимостью,195255.623276
13740,0,NaN,0,среднее,1,гражданский брак,1,F,сотрудник,0,NaN,на проведение свадьбы,142763.314708
19828,0,NaN,0,среднее,1,женат / замужем,0,F,сотрудник,0,NaN,жилье,142763.314708


Возникновение пропусков связано с тем, что при заполнении пропусков средним для данного ПОЛА и ВОЗРАСТА нулевые значения возраста игнорировались по причине неприменимости использованной группировки к заемщиками с фактически неуказанным возрастом.  
  
  
Пропуски в колонке 'total_income' будут заполнены средними значениями доходов для заемщиков данного пола с учетом уже проведенного обогащения данных. 

In [12]:
total_income_mean_gender = data_raw.groupby('gender').agg({'total_income':'mean'}).rename(columns={'total_income': 'total_income_mean_gender'})
data_raw = data_raw.merge(total_income_mean_gender, on='gender', how='left')
data_raw.loc[data_raw.loc[:, 'total_income'].isna(), 'total_income'] = data_raw.loc[data_raw.loc[:, 'total_income'].isna(), 'total_income_mean_gender']
data_raw.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21524 entries, 0 to 21523
Data columns (total 14 columns):
children                    21524 non-null int64
days_employed               19350 non-null float64
dob_years                   21524 non-null int64
education                   21524 non-null object
education_id                21524 non-null int64
family_status               21524 non-null object
family_status_id            21524 non-null int64
gender                      21524 non-null object
income_type                 21524 non-null object
debt                        21524 non-null int64
total_income                21524 non-null float64
purpose                     21524 non-null object
total_income_mean           21524 non-null float64
total_income_mean_gender    21524 non-null float64
dtypes: float64(4), int64(5), object(5)
memory usage: 2.5+ MB


#### Промежуточный вывод
Пропуски в столбце 'total_income' были заполнены средними для данного пола и возраста значениями.  
В случаях, когда ежемесячный доход не был указан у заемщиков с нулевым возрастом, пропуски в данной колонке были заполнены средними для соответствющего пола.

In [13]:
def replace_na_days_employed_with_median(df):
    """
    Функция 1) заменяет данные о стаже на аналогичные взятые по модулю, 
            2) собирает из входного датасета данные о выбросах в колонке "стаж" (days_employed). Выбросом считается значение 
               стажа, превышающее возраст заемщика. Случаи, когда стаж превышает значение (возраст заемщика "минус" 14 лет) 
               игнорируются: ошибка принимается за случайную, допущенную при ручном внесении данных в систему,
            3) заполняет пропуски медианными значениями стажа для группы, полученой в результате группировки сначала по полу,
               затем по возрасту
    
    :df: исследуемый датасет
    :outlier_days_employed: датасет, содержащий в себе все строки из df, в которых стаж превышает текущий возраст. Содержит 
                            колонки 'dob_years', 'days_employed', 'years_excess' (превышение стажа над возрастом в годах)
    :outlier_days_employed_grouped: outlier_days_employed, сгруппированный сначала по полу, затем по возрасту. Колонка
                                    'days_employed_median' содержит медианные для каждой группы значения стажа
    :df_merged: итоговый датасет, получаемый путем последовательного слияния исходного df с промежуточными таблицами
                (последовательность слияний см. ниже)
    :return: кортеж из трех датасетов: 
             1) исходный датасет, в котором пропуски в 'days_employed' заменены на медианное для данного пола и возраста,
                аномальные значения стажа также заменены на медианное для данной группы. Стаж во всех случаях взят по модулю
             2) датасет, содержащий колонки 'dob_years' - возраст заемщика, 'days_employed' - заявленный стаж заемщика,
                'years_excess' - превышение стажа над возрастом в годах. Нумерация строк совпадает с нумерацией в исходном
                датасете. 
    """
    # приведение всех данных о стаже к неотрицательным значениям
    df['days_employed'] = abs(df['days_employed'])
    
    # создание датасета с аномальным стажем (содержание см. в описании функции)
    outlier_days_employed = df.loc[df.loc[:, 'dob_years'] * 365 - df.loc[:, 'days_employed'] < 0][['dob_years', 'days_employed']]
    
    # создание колонки с превышением стажа над возрастом в годах
    outlier_days_employed['years_excess'] = outlier_days_employed['days_employed'] / 365 - outlier_days_employed['dob_years']
    
    # создание датасета (содержание см. в описании функции)
    no_outlier_days_employed = df.loc[~(df.loc[:, 'days_employed'].isna())&(df.loc[:, 'dob_years'] * 365 - df.loc[:, 'days_employed'] >= 0)]
    
    # группировка датасета по полу и возрасту, подсчет медианных значений стажа
    no_outlier_days_employed_grouped = no_outlier_days_employed.groupby(['gender', 'dob_years']).agg({'days_employed': 'median'}).rename(columns={'days_employed': 'days_employed_median'})
    
    # присоединение к исходному датасету колонки с данными о среднем стаже
    df_merged = df.merge(no_outlier_days_employed_grouped, on=['gender', 'dob_years'], how='left')
    
    # замена пропусков в колонке 'days_employed' на медианные значения стажа 
    df_merged.loc[(df_merged.loc[:, 'days_employed'].isna()), 'days_employed'] = df_merged.loc[df_merged.loc[:, 'days_employed'].isna(), 'days_employed_median']
    
    return df_merged, outlier_days_employed

data_raw, outlier_days_employed = replace_na_days_employed_with_median(data_raw)
data_raw.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21524 entries, 0 to 21523
Data columns (total 15 columns):
children                    21524 non-null int64
days_employed               21514 non-null float64
dob_years                   21524 non-null int64
education                   21524 non-null object
education_id                21524 non-null int64
family_status               21524 non-null object
family_status_id            21524 non-null int64
gender                      21524 non-null object
income_type                 21524 non-null object
debt                        21524 non-null int64
total_income                21524 non-null float64
purpose                     21524 non-null object
total_income_mean           21524 non-null float64
total_income_mean_gender    21524 non-null float64
days_employed_median        21419 non-null float64
dtypes: float64(5), int64(5), object(5)
memory usage: 2.6+ MB


#### Промежуточный вывод
После проведенных операций в колонке 'days_employed' все еще остаются пропуски. Необходима дальнейшая проверка.  

In [14]:
days_employed_na_count = data_raw.loc[(data_raw.loc[:, 'days_employed'].isna())]['gender'].count()
print(f'Количество строк с пропусками: {days_employed_na_count}')

Количество строк с пропусками: 10


Строки с пропусками в колонке 'days_employed':

In [15]:
data_raw.loc[(data_raw.loc[:, 'days_employed'].isna())].head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,total_income_mean,total_income_mean_gender,days_employed_median
1890,0,NaN,0,высшее,0,Не женат / не замужем,4,F,сотрудник,0,154137.194106,жилье,142763.314708,154137.194106,NaN
2284,0,NaN,0,среднее,1,вдовец / вдова,2,F,пенсионер,0,154137.194106,недвижимость,142763.314708,154137.194106,NaN
4064,1,NaN,0,среднее,1,гражданский брак,1,M,компаньон,0,193077.248413,ремонт жилью,195255.623276,193077.248413,NaN
5014,0,NaN,0,среднее,1,женат / замужем,0,F,компаньон,0,154137.194106,покупка недвижимости,142763.314708,154137.194106,NaN
6411,0,NaN,0,высшее,0,гражданский брак,1,F,пенсионер,0,154137.194106,свадьба,142763.314708,154137.194106,NaN
6670,0,NaN,0,Высшее,0,в разводе,3,F,пенсионер,0,154137.194106,покупка жилой недвижимости,142763.314708,154137.194106,NaN
8574,0,NaN,0,среднее,1,женат / замужем,0,F,сотрудник,0,154137.194106,недвижимость,142763.314708,154137.194106,NaN
12402,3,NaN,0,среднее,1,женат / замужем,0,M,сотрудник,0,193077.248413,операции с коммерческой недвижимостью,195255.623276,193077.248413,NaN
13740,0,NaN,0,среднее,1,гражданский брак,1,F,сотрудник,0,154137.194106,на проведение свадьбы,142763.314708,154137.194106,NaN
19828,0,NaN,0,среднее,1,женат / замужем,0,F,сотрудник,0,154137.194106,жилье,142763.314708,154137.194106,NaN


Возникновение этих строк связано с тем, что в изначальном датасете в этих строках в колонке 'days_employed' был пропуск, а в колонке 'dob_years' был указан 0.  
Пропуски будут удалены, в силу того, что без дополнительного анализа невозможно однозначно сделать выбор в пользу медианы или среднего значения (об этом далее). 

In [16]:
#удаление оставшихся 10 пропусков и вывод информации о текущем состоянии таблицы
# df - исходный датасет после очистки от пропусков
df = data_raw.dropna().reset_index(drop=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21419 entries, 0 to 21418
Data columns (total 15 columns):
children                    21419 non-null int64
days_employed               21419 non-null float64
dob_years                   21419 non-null int64
education                   21419 non-null object
education_id                21419 non-null int64
family_status               21419 non-null object
family_status_id            21419 non-null int64
gender                      21419 non-null object
income_type                 21419 non-null object
debt                        21419 non-null int64
total_income                21419 non-null float64
purpose                     21419 non-null object
total_income_mean           21419 non-null float64
total_income_mean_gender    21419 non-null float64
days_employed_median        21419 non-null float64
dtypes: float64(5), int64(5), object(5)
memory usage: 2.5+ MB


В функции *replace_na_days_employed_with_median* также были определены выбросы в колонке 'days_employed'. Строки содержащие выбросы были сохранены в отдельный датафрэйм. Именно выбросы стали причиной удаления 10 строк таблицы пунктом выше.   
Сводная информация о строках с выбросами:

In [17]:
outlier_days_employed.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3519 entries, 4 to 21520
Data columns (total 3 columns):
dob_years        3519 non-null int64
days_employed    3519 non-null float64
years_excess     3519 non-null float64
dtypes: float64(2), int64(1)
memory usage: 110.0 KB


Первые 5 строк (в колонке 'years_excess' приведено превышение стажа над возрастом в годах):

In [18]:
outlier_days_employed.head(10)

,dob_years,days_employed,years_excess
4,53,340266.072047,879.235814
18,53,400281.136913,1043.660649
24,57,338551.952911,870.539597
25,67,363548.489348,929.023258
30,62,335581.668515,857.401832
35,68,394021.072184,1011.509787
50,63,353731.432338,906.127212
56,64,370145.087237,950.096129
71,62,338113.529892,864.338438
78,61,359722.945074,924.542315


In [19]:
outlier_days_employed_percent = outlier_days_employed.shape[0] / df.shape[0]
print("Процент строк, содержащих выбросы в колонке 'days_employed': {:.1%}".format(outlier_days_employed_percent))

Процент строк, содержащих выбросы в колонке 'days_employed': 16.4%


В 16 процентах случаев данные в колонке 'days_employed' аномально завышены. В ТЗ указаны единицы измерения трудового стажа - дни, однако такое превышение говорит об иных единицах измерения. Систематический характер ошибки явно указывает на сбой (в рамках данного ТЗ) в алгоритмах получения данных.  
Вероятнее всего алгоритм в 16.4% случаев регистрирует стаж в часах. С учетом этого превышение устраняется:

In [20]:
# в колонке 'start_career' указывается возраст заемщика, с которого начинается отчет его трудового стажа
outlier_days_employed['career_start'] = (outlier_days_employed['days_employed']) / (24 * 365)
outlier_days_employed.head(10)

,dob_years,days_employed,years_excess,career_start
4,53,340266.072047,879.235814,38.843159
18,53,400281.136913,1043.660649,45.694194
24,57,338551.952911,870.539597,38.647483
25,67,363548.489348,929.023258,41.500969
30,62,335581.668515,857.401832,38.308410
35,68,394021.072184,1011.509787,44.979574
50,63,353731.432338,906.127212,40.380300
56,64,370145.087237,950.096129,42.254005
71,62,338113.529892,864.338438,38.597435
78,61,359722.945074,924.542315,41.064263


Необходимо выяснить, присутствуют ли с учетом введенных поправок на единицы измерения строки, в которых стаж все равно превышает возраст:

In [21]:
outlier_days_employed.loc[outlier_days_employed.loc[:, 'dob_years'] - outlier_days_employed.loc[:, 'career_start'] <= 0].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 145 entries, 99 to 21312
Data columns (total 4 columns):
dob_years        145 non-null int64
days_employed    145 non-null float64
years_excess     145 non-null float64
career_start     145 non-null float64
dtypes: float64(3), int64(1)
memory usage: 5.7 KB


В 145 случаях стаж превышает возраст: это может быть связано как с указанием нулевого возраста, так и с указанием ненулевого возраста, который тем не менее не превышает стаж.  
Для получения более полного представления о механизмах возникновения подобной проблемы был проведен расчет среднего возраста заёмщиков, данные о стаже которых были внесены некорректно:

In [22]:
mean_dob_years = outlier_days_employed['dob_years'].mean()
min_dob_years = outlier_days_employed['dob_years'].min()
max_dob_years = outlier_days_employed['dob_years'].max()
print('Средний возраст заёмщиков, данные о стаже которых были внесены некорректно: {:.1f}'.format(mean_dob_years))
print('Минимальный возраст заёмщиков, данные о стаже которых были внесены некорректно: {:.1f}'.format(min_dob_years))
print('Максимальный возраст заёмщиков, данные о стаже которых были внесены некорректно: {:.1f}'.format(max_dob_years))

Средний возраст заёмщиков, данные о стаже которых были внесены некорректно: 57.9
Минимальный возраст заёмщиков, данные о стаже которых были внесены некорректно: 0.0
Максимальный возраст заёмщиков, данные о стаже которых были внесены некорректно: 74.0


Утверждение о наличии превышения стажа над возрастом в случае, когда возраст равен 0 подтвердилось, однако среднее значение возраста равно 57.9, значит, проблема наиболее часто возникает в случае более старших заёмщиков. Эти выводы могут помочь с устранением ошибки.

### Вывод

1. Данные о стаже были приведены к неотрицательному виду
1. На данном этапе работы сначала было установлено общее количество пропусков. Значения отсутствовали в колонках 'days_employed' и 'total_income'. 
2. Анализ показал, что во всех случаях пропуски "идут парами", что явно указывает на системный характер их возникновения.  
3. При заполнении пропусков использовались статистические характеристики групп, полученных путем разделения данных по полу, затем по возрасту.  
4. Пропуски в колонке 'total_income' были заполнены средними значенями для группы, в случае, если группировка по возрасту неприменима, группировка производилась только по половому признаку
5. Пропуски в колонке 'days_employed' были заполнены медианными значениями для данной группы
6. На этапе заполнения пропусков были удалены следующие строки: 
  1. Строка со значением 'XNA' в колонке 'gender'
  2. 10 строк, в которых в колонке 'days_employed' стоят пропуски, а возраст равен 0
7. Была обнаружена систематическая ошибка в колонке 'days_employed': в 16.4% случаев стаж указывался в часах. В измененном датасете сохранилос 145 строк, в которых стаж превышает возраст. Были установлены базовые характеристики данных с ошибками, которые могут быть использованы при анализе проблемы
8. В результате обогащения данных датасет сократился на 0.05%, вместо 10.1%

### Обработка оставшихся выбросов

Все колонки с количественными значениями следует проверить на наличие выбросов - аномально высоких значений.  
Ранее это уже было сделано с колонкой 'days_employed', выяснилось, что в 16% случаев значение стажа, предположительно, записано в часах (подробности - в выводах выше).  
Также на этапе группировки данных по возрасту и полу было установлено, что выбросы присутствуют в колонке 'dob_years' (речь о нулевом возрасте - отрицательных или экстримально высоких значений нет). Присутствие этих выбросов было учтено при заполнении пропусков.  
Отсутствие выбросов в колонке 'total_income' было доказано выше.  
Необходимо проверить на наличие выбросов колонки 'children', 'debt'.  
  
#### Колонка 'children'

In [23]:
# группировка датасета по количеству детей
df_grouped = df.groupby('children')['dob_years'].count()
df_grouped

children
-1        47
 0     14076
 1      4801
 2      2042
 3       328
 4        41
 5         9
 20       75
Name: dob_years, dtype: int64

В данных присутствуют строки с отрицательным количеством детей, и данные о заемщиках с 20-ю детьми.  
1. В первом случае необходимо заменить -1 на 1: принимается, что это ошибка ввода, а не условное обозначение отстутствия детей. В любом случае, группа с единственным ребенком расширится всего на 1%, что не должно негативно сказаться на итоговом значении соответствующей метрики.  
2. Во втором случае 20 будет заменено на 2: очевидно, что ошибка возникла при ручном вводе числа 2, расположенного на клавиатуре над числом 0. 

In [24]:
# замена значений -1 в колонке 'children'
df.loc[df['children'] == -1, 'children'] = df.loc[df['children'] == -1, 'children'].replace(-1, 1)

# замена значений 20 в колонке 'children'
df.loc[df['children'] == 20, 'children'] = df.loc[df['children'] == 20, 'children'].replace(20, 2)

# повторная группировка по столбцу 'children' и вывод результата
df_grouped = df.groupby('children')['dob_years'].count()
df_grouped

children
0    14076
1     4848
2     2117
3      328
4       41
5        9
Name: dob_years, dtype: int64

Выбросы в колонке 'children' устранены

#### Колонка 'debt'

В колонке 'debt' использован логический тип данных.  
Проверка колонки на наличие аномальных значений:

In [25]:
debt_values = df['debt'].unique()
print(f'Значения в колонке debt: {debt_values}')

Значения в колонке debt: [0 1]


Данные в колонке 'debt' корректны

#### Колонка 'total_income'

In [26]:
total_income_max = df['total_income'].max()
total_income_min = df['total_income'].min()
total_income_mean = df['total_income'].mean()
print(f'Минимальное значение дохода: {total_income_min}')
print(f'Максимальное значение дохода: {total_income_max}')
print(f'Среднее значение дохода: {total_income_mean}')

Минимальное значение дохода: 20667.26379327158
Максимальное значение дохода: 2265604.028722744
Среднее значение дохода: 167371.64669376472


Основываясь на анализе статистических показателей колонки 'total_income', можно сделать вывод об отсутствии выбросов: минимальный доход неотрицателен, максимальный - относительно высок, но превышает минимальный всего на два порядка. 

#### Вывод
К данному моменту на наличие выбросов были проверены все колонки с числовыми данными, имеющиеся выбросы были устранены.

### Замена типа данных

В колонках 'days_employed' и 'total_income' данные представлены типом float64. Необходимо провести замену на int64 с целью повышения точности анализа и упрощения расчетов. 

In [27]:
# замена типа данных в колонке 'days_employed'
df['days_employed'] = df['days_employed'].astype('int')

# замена типа данных в колонке 'total_income'
df['total_income'] = df['total_income'].astype('int')

# вывод информации о таблице
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21419 entries, 0 to 21418
Data columns (total 15 columns):
children                    21419 non-null int64
days_employed               21419 non-null int64
dob_years                   21419 non-null int64
education                   21419 non-null object
education_id                21419 non-null int64
family_status               21419 non-null object
family_status_id            21419 non-null int64
gender                      21419 non-null object
income_type                 21419 non-null object
debt                        21419 non-null int64
total_income                21419 non-null int64
purpose                     21419 non-null object
total_income_mean           21419 non-null float64
total_income_mean_gender    21419 non-null float64
days_employed_median        21419 non-null float64
dtypes: float64(3), int64(7), object(5)
memory usage: 2.5+ MB


### Вывод


1. Замена типов данных была проведена при помощи метода *astype*.  
2. Данный способ был выбран в силу того, что в исходном состоянии данные представлены в формате float64 и применение функции *to_numeric* невозможно.  
3. Замена типов данных в колонках 'days_employed' и 'total_income' не привела к возникновению новых пропусков.

### Обработка дубликатов

Информация о текущем состоянии таблицы:

In [28]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21419 entries, 0 to 21418
Data columns (total 15 columns):
children                    21419 non-null int64
days_employed               21419 non-null int64
dob_years                   21419 non-null int64
education                   21419 non-null object
education_id                21419 non-null int64
family_status               21419 non-null object
family_status_id            21419 non-null int64
gender                      21419 non-null object
income_type                 21419 non-null object
debt                        21419 non-null int64
total_income                21419 non-null int64
purpose                     21419 non-null object
total_income_mean           21419 non-null float64
total_income_mean_gender    21419 non-null float64
days_employed_median        21419 non-null float64
dtypes: float64(3), int64(7), object(5)
memory usage: 2.5+ MB


Необходимо привести к нижнему регистру все строки, содержащие тип данных object. Это также касается столбца 'gender', в котором, как стало известно выше все обозначения представлены в едином формате (F и M), это необходимо для единой стилистики заполнения данных.

In [29]:
# приведение к нижнему регистру содержания всех колонок, которые будут использованы при дальнейшем анализе
df['education'] = df['education'].str.lower()
df['family_status'] = df['family_status'].str.lower()
df['gender'] = df['gender'].str.lower()
df['income_type'] = df['income_type'].str.lower()
df['purpose'] = df['purpose'].str.lower()

# вывод первых 5 и последних 5 строк датасета
df

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,total_income_mean,total_income_mean_gender,days_employed_median
0,1,8437,42,высшее,0,женат / замужем,0,f,сотрудник,0,253875,покупка жилья,156472.182083,154137.194106,2317.699323
1,1,4024,36,среднее,1,женат / замужем,0,f,сотрудник,0,112080,приобретение автомобиля,167336.145990,154137.194106,1863.725938
2,0,5623,33,среднее,1,женат / замужем,0,m,сотрудник,0,145885,покупка жилья,211319.103292,193077.248413,1266.427717
3,3,4124,32,среднее,1,женат / замужем,0,m,сотрудник,0,267628,дополнительное образование,197448.674184,193077.248413,1298.674163
4,0,340266,53,среднее,1,гражданский брак,1,f,пенсионер,0,158616,сыграть свадьбу,156204.827930,154137.194106,2575.811475
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21414,1,4529,43,среднее,1,гражданский брак,1,f,компаньон,0,224791,операции с жильем,170624.375174,154137.194106,2137.120412
21415,0,343937,67,среднее,1,женат / замужем,0,f,пенсионер,0,155999,сделка с автомобилем,126146.856577,154137.194106,1770.530419
21416,1,2113,38,среднее,1,гражданский брак,1,m,сотрудник,1,89672,недвижимость,198932.976537,193077.248413,1623.108469
21417,3,3112,38,среднее,1,женат / замужем,0,m,сотрудник,1,244093,на покупку своего автомобиля,198932.976537,193077.248413,1623.108469


Поиск полных дупликатов:

In [30]:
# подсчет количества полных дупликатов до очистки
complete_duplicates = df.duplicated().sum()
print(f'Количество полных дупликатов до очистки: {complete_duplicates}')

# удаление дупликатов и сброс индексов
df = df.drop_duplicates().reset_index(drop='True')
complete_duplicates = df.duplicated().sum()
print(f'Количество полных дупликатов после очистки: {complete_duplicates}')

Количество полных дупликатов до очистки: 71
Количество полных дупликатов после очистки: 0


Замаскированные дупликаты искались среди значений в колонках, не требующих стемминга: маловероятно, что совпадение всех данных подобного типа о заемщике связано с реальной ситуацией (один человек берет несколько кредитов в один день в одном и том же банке).  
Удостоверимся в пригодности колонок 'education' и 'family_status_id' для фильтрации путем подсчета количества их различных значений.

In [31]:
df['education'].value_counts()

среднее                15105
высшее                  5214
неоконченное высшее      741
начальное                282
ученая степень             6
Name: education, dtype: int64

In [32]:
df['family_status_id'].value_counts()

0    12287
1     4128
4     2794
3     1185
2      954
Name: family_status_id, dtype: int64

В колонке 'education' отсутствуют одинаковые по смыслу значения, записанные разными способами: колонка пригодна для поиска дупликатов.  
ID семейного положение также может быть применено в поиске дупликатов, по той же причине.

In [33]:
partial_duplicates = df.duplicated(subset=['children', 'days_employed', 'dob_years', 'family_status_id',
                                           'debt', 'total_income', 'gender', 'education']).sum()
print(f'Количество замаскированных дупликатов: {partial_duplicates}')

Количество замаскированных дупликатов: 1019


Скрытыми дупликатами выше признаны данные о заемщиках, которые совпадают по количеству детей, стажу, возрасту, семейному положению, факту наличия задолженности, ежемесячному доходу, полу и уровню образования. Существование двух разных заемщиков с настолько схожим набором характеристик крайне маловероятно.  
Скрытые дупликаты будут удалены:

In [34]:
df = df.drop_duplicates(subset=['children', 'days_employed', 'dob_years', 'family_status_id',
                                'debt', 'total_income', 'gender', 'education']).reset_index(drop='True')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20329 entries, 0 to 20328
Data columns (total 15 columns):
children                    20329 non-null int64
days_employed               20329 non-null int64
dob_years                   20329 non-null int64
education                   20329 non-null object
education_id                20329 non-null int64
family_status               20329 non-null object
family_status_id            20329 non-null int64
gender                      20329 non-null object
income_type                 20329 non-null object
debt                        20329 non-null int64
total_income                20329 non-null int64
purpose                     20329 non-null object
total_income_mean           20329 non-null float64
total_income_mean_gender    20329 non-null float64
days_employed_median        20329 non-null float64
dtypes: float64(3), int64(7), object(5)
memory usage: 2.3+ MB


In [35]:
# процент дупликатов в очищенном от пропусков датасете
duplicates_percent = (complete_duplicates + partial_duplicates) / 21419
print('Процент удаленных дупликатов в очищенной от пропусков таблице: {:.1%}'.format(duplicates_percent))

# общее сокращение объема данных в результате их подготовки
volume_reduce_percent = (21525 - 20329) / 21525
print('Общее сокращение объема данных в результате их подготовки: {:.1%}'.format(volume_reduce_percent))

Процент удаленных дупликатов в очищенной от пропусков таблице: 4.8%
Общее сокращение объема данных в результате их подготовки: 5.6%


### Вывод

Была проведена очистка данных от дупликатов:
1. Количество полных дупликатов составило 71
2. Банки, как правило не выдают одному и тому же человеку несколько кредитов в день. Потому за скрытые дупликаты принимались данные о заёмщиках с одновременно одинаковыми стажем, количеством детей, возрастом, семейным положением, фактом наличия задолженности, ежемесячным доходом, полом и уровнем образования.  
  
Причинами возникновения полных дупликатов могли послужить ошибки в алгоритмах выгрузки данных из базы или, если данные были зашифрованы, ошибки в алгоритмах дешифровки.  
Неполные дубликаты могли возникнуть по причине использования различных формулировок в колонке 'purpose' при переносе данных с печатных носителей на электронные в ручном режиме.  
  
___Итоговое сокращение объема данных составило 5.6%___

### Лемматизация

Наиболее часто упоминаемые слова в колонке 'purpose' определены по следующему алгоритму:

In [36]:
m = Mystem()

# получен массив уникальных значений в колонке 'purpose'
purpose_content = df['purpose'].unique()

# уникальные значения из 'purpose' собраны в список
purpose_content_str = ' '.join(df['purpose'].unique())

# список лемматизирован
purpose_content_str_lemmas = m.lemmatize(purpose_content_str)

# слова из списка выведены в порядке убывания
Counter(purpose_content_str_lemmas).most_common(100)

[(' ', 96),
 ('покупка', 10),
 ('недвижимость', 10),
 ('автомобиль', 9),
 ('образование', 9),
 ('жилье', 7),
 ('с', 5),
 ('операция', 4),
 ('на', 4),
 ('свой', 4),
 ('свадьба', 3),
 ('строительство', 3),
 ('получение', 3),
 ('высокий', 3),
 ('дополнительный', 2),
 ('для', 2),
 ('коммерческий', 2),
 ('жилой', 2),
 ('подержать', 2),
 ('заниматься', 2),
 ('сделка', 2),
 ('приобретение', 1),
 ('сыграть', 1),
 ('проведение', 1),
 ('семья', 1),
 ('собственный', 1),
 ('со', 1),
 ('профильный', 1),
 ('сдача', 1),
 ('ремонт', 1),
 ('\n', 1)]

Анализ статистики упоминания слов в колонке 'purpose' явно указывает на следующие цели получения кредита: 
1. Операции с недвижимостью
2. Операции с транспортом
3. Образование
4. Свадьба  
  
Дальнейшая лемматизация будет произведена по следующему итеративному алгоритму:
1. Все строки, содержащие слова 'жилье', 'недвижимость', 'автомобиль', 'образование' будут отнесены к соответсвующей категории из списка выше
2. Данным, которые не попадут ни в одну из категорий, будет присвоено значение 'неизвестная цель'
3. Каждый из случаев 'неизвестной цели' будет изучен и отнесен к одной из уже существующих категорий, либо будет создана новая категория

In [37]:
def purpose_lemmas(df):
    """
    Функция присваивает каждой строке таблицы группу целей взятия кредита, основываясь на результатах лемматизации, проведенной
    ранее. Все записи распределяются на 4 группы, указаные выше. 
    
    :df - исходный датасет
    :return - исходный датасет с колонкой 'purpose_type', содержащей информацию о группе целей
    """
    m = Mystem()

    def make_lemma_column(lemmas):
        """
        Функция относит строку к одной из выделенных групп целей
        
        :lemmas - лемматизированное содержание колонки 'purpose'(ячейка)
        :return - строка с названием группы
        """
        if 'жилье' in lemmas:
            return 'операции с недвижимостью'
        elif 'недвижимость' in lemmas:
            return 'операции с недвижимостью'
        elif 'образование' in lemmas:
            return 'получение образования'
        elif 'свадьба' in lemmas:
            return 'организация свадьбы'
        elif 'автомобиль' in lemmas:
            return 'операции с транспортом'
        else:
            return 'неизвестная цель'
        
    df['purpose_lemmas'] = df['purpose'].apply(m.lemmatize)
    df['purpose_type'] = df['purpose_lemmas'].apply(make_lemma_column)
    return df


df = purpose_lemmas(df)
unknown_purpose_type = df.loc[(df.loc[:, 'purpose_type'] == 'неизвестная цель')]['debt'].count()
print(f'Количество строк с неизвестной целью: {unknown_purpose_type}')

Количество строк с неизвестной целью: 0


### Вывод

В результате применения указанного выше алгоритма данные были разбиты по целям займов в полном объеме с первой итерации, что говорит о том, что список целей, сформулированный на основе частоты упоминаемости слов, оказался достаточным. 

### Категоризация данных

In [38]:
def children_group(df):
    """
    Функция разбивает данные на категории по факту наличия детей
    :df - исходный датасет
    :return - df_grouped_children - распределение количества должников среди заемщиков с детьми и без них
            - df_grouped_children - распределение количества должников среди заемщиков с разным количеством детей
    """
    
    def make_got_children_column(row):
        """
        Функция одной строки: определяет, есть ли у заемщика дети
        
        row: строка исходной таблицы
        return: строка, содержащая информацию о наличии/отсутствии детей
        """
        if row['children'] != 0:
            return 'got kids'
        else:
            return 'no kids'
        
    df['got_children'] = df.apply(make_got_children_column, axis=1)
    df_grouped_children = df.groupby(['got_children'])['debt', 'gender'].agg({'debt': 'sum', 'gender': 'count'}).rename(columns={'debt': 'got_debt', 'gender': 'total_clients'})
    df_grouped_children['debt_percent'] = df_grouped_children['got_debt'] * 100 / df_grouped_children['total_clients']
    df_grouped_children['debt_percent'] = df_grouped_children['debt_percent']
    
    df_grouped_children2 = df.groupby(['children'])['debt', 'gender'].agg({'debt': 'sum', 'gender': 'count'}).rename(columns={'debt': 'got_debt', 'gender': 'total_clients'})
    df_grouped_children2['debt_percent'] = df_grouped_children2['got_debt'] * 100 / df_grouped_children2['total_clients']
    df_grouped_children2['debt_percent'] = df_grouped_children2['debt_percent']
    return df_grouped_children, df_grouped_children2[1:].sort_values(by='debt_percent', ascending=False)


df_grouped_children, df_grouped_children2 = children_group(df)

In [39]:
def family_group(df):
    """
    Функция категоризирует данные по семейному положению
    :df - исходный датасет
    :return - исходный датасет, категоризированный по семейному положению и отсортированный по мере возрастания количества
              должников в группе
    """
    df_grouped_family = df.groupby('family_status')['debt', 'gender'].agg({'debt': 'sum', 'gender': 'count'}).rename(columns={'gender': 'total_clients'})
    df_grouped_family['debt_percent'] = df_grouped_family.loc[:, 'debt'] * 100 / df_grouped_family.loc[:, 'total_clients']
    return df_grouped_family.sort_values(by='debt_percent', ascending=False)

df_grouped_family = family_group(df)

In [40]:
def income_group(df):
    """
    Функция распределяет заемщиков на 11 групп в зависимости от уровня их дохода
    df - исходный датасет
    return - сводная таблица на основе исходного датасета, данные распределены по уровню дохода заемщиков, для каждой группы
             вычислен процент должников
    """
    
    def total_income_group(row):
        """
        Функция одной строки: присваивает заемщику группу дохода на основе значения в колонке 'total_income'
        row - строка исходного датасета
        return - группа дохода данного заемщика
        """
        try:
            if row['total_income'] <= 0:
                return 'неположительный доход'
            elif 0 < row['total_income'] < 50000:
                return 'менее 50 тыс.'
            elif 50000 <= row['total_income'] < 100000:
                return 'от 50 до 100 тыс.'
            elif 100000 <= row['total_income'] < 150000:
                return 'от 100 до 150 тыс.'
            elif 150000 <= row['total_income'] < 200000:
                return 'от 150 до 200 тыс.'
            elif 200000 <= row['total_income'] < 250000:
                return 'от 200 до 250 тыс.'
            elif 250000 <= row['total_income'] < 300000:
                return 'от 250 до 300 тыс.'
            elif 300000 <= row['total_income'] < 350000:
                return 'от 300 до 350 тыс.'
            elif 350000 <= row['total_income'] < 400000:
                return 'от 350 до 400 тыс.'
            elif 400000 <= row['total_income'] < 450000:
                return 'от 400 до 450 тыс.'
            elif 450000 <= row['total_income'] < 500000:
                return 'от 450 до 500 тыс.'
            elif row['total_income'] > 500000:
                return 'более 500 тыс.'
        except:
            return 'неучтенный уровень дохода'
        
    df['total_income_group'] = df.apply(total_income_group, axis=1)
    df_income_grouped_income = df.groupby('total_income_group')['debt', 'gender'].agg({'debt': 'sum', 'gender': 'count'}).rename(columns={'gender': 'total_clients'})
    df_income_grouped_income['debt_percent'] = df_income_grouped_income['debt'] * 100 / df_income_grouped_income['total_clients']
    return df_income_grouped_income.sort_values(by='debt_percent', ascending=False)


df_grouped_income = income_group(df)

In [41]:
def purpose_group(df):
    """
    Функция распределяет заемщиков по типам целей взятия кредита на 4 выделенных выше группы
    :df - исходный датасет
    :return - сводная таблица по типу цели с указанием процента должников внутри каждой группы
    """
    df_grouped_purpose = df.groupby('purpose_type')['debt', 'gender'].agg({'debt': 'sum', 'gender': 'count'}).rename(columns={'gender': 'total_clients'})
    df_grouped_purpose['debt_percent'] = df_grouped_purpose['debt'] * 100 / df_grouped_purpose['total_clients']
    return df_grouped_purpose.sort_values(by='debt_percent', ascending=False)


df_grouped_purpose = purpose_group(df)

### Вывод

Данные были категоризированы в строгом соответствии с поставленными вопросами.  
Во всех случаях данные:  
1. Разбивались на группы методом .groupby()
2. В каждой группе подсчитывалось количество заемщиков, имеющих задолженности по кредиту
3. Расчитывался процент должников относительно общего числа заемщиков в группе
4. Итоговая таблица сортировалась в порядке убывания по колонке, содержащей данные о проценте должников в группе

### Оценка влияния свойств заемщиков на платежеспособность
Во всех сводных таблицах чем выше позиция, тем больше должников (меньше надежность группы)

- Есть ли зависимость между наличием детей и возвратом кредита в срок?  
  1. Сводная таблица по факту наличия детей и наличию задолженности 
    - got_children - факт наличия детей
    - got_debt - число заемщиков, имеющих задолженности
    - total_clients - общее число заемщиков в группе
    - debt_percent - процент заемщиков-должников в группе

In [42]:
df_grouped_children

,got_debt,total_clients,debt_percent
got_children,,,
got kids,672,7064,9.513024
no kids,1045,13265,7.877874


Согласно данным в этой сводной таблице, заемщики, у которых есть дети, имеют задолженности на 1.63% чаще.  
_Заемщик с детьми платит по кредиту хуже_

Проанализируем, как количество детей влияет на наличие задолженностей:  

In [43]:
df_grouped_children2

,got_debt,total_clients,debt_percent
children,,,
4,4,40,10.000000
2,201,2043,9.838473
1,440,4648,9.466437
3,27,324,8.333333
5,0,9,0.000000


Если считать выборки внутри групп репрезентативными (даже выборки для заемщиков с 4 и 5 детьми) и игнорировать перекрестное влияние различных факторов на факт наличия задолженности, можно сделать следующие выводы:  
1. _Хуже всех по долгам платят заемщики с 4 детьми (10% случаев)_  
2. _Лучше всех по долгам платят заемщики с 5 детьми_
3. _Разница по надежности заемщиков с 2 детьми и заемщиков с 1 ребенком почти отсутствует (9.84% против 9.47%)_

### Вывод

1. Анализ данной выборки показал, что __заемщики с детьми платят__ по кредитам __хуже__.  
2. С учетом приведенных выше допущений, среди заемщиков с детьми самыми надежными являются заемщики с 5 детьми, самыми ненадежными - заемщики с 4 детьми. 

- Есть ли зависимость между семейным положением и возвратом кредита в срок?  
Сводная таблица по семейному положению и наличию задолженностей: 
  - family_status - семейное положение
  - debt - количество заемщиков-должников в группе
  - total_clients - общее количество заемщиков в группе
  - debt_percent - процент клиентов с задолженностями в группе

In [44]:
df_grouped_family

,debt,total_clients,debt_percent
family_status,,,
не женат / не замужем,272,2706,10.051737
гражданский брак,383,3983,9.615867
женат / замужем,915,11568,7.909751
в разводе,85,1162,7.314974
вдовец / вдова,62,910,6.813187


### Вывод

Согласно полученным результатам: 
1. Хуже всего по долгам платят заемщики, не построившие семьи
2. Лучше всего по долгам платят заемщики, потерявшие своих супругов (вдовцы и вдовы)
3. Заемщики, состоящие в гражданском браке, являются заметно менее надежными, нежели заемщики, чей брак оформлен официально (7.91% против 9.62%, разница - 1.71%)

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?  
  Сводная таблица по уровню дохода и факту наличия задолженности
  - total_income_group - группа дохода (11 групп от 0 до 500+ тыс. с шагом в 50 тыс.)
  - debt - количество заемщиков-должников в группе
  - total_clients - общее количество заемщиков в группе
  - debt_percent - процент клиентов с задолженностями в группе

In [45]:
df_grouped_income

,debt,total_clients,debt_percent
total_income_group,,,
от 150 до 200 тыс.,466,4761,9.787860
от 100 до 150 тыс.,518,5934,8.729356
от 300 до 350 тыс.,51,621,8.212560
от 50 до 100 тыс.,330,4070,8.108108
от 200 до 250 тыс.,187,2395,7.807933
от 350 до 400 тыс.,24,329,7.294833
от 400 до 450 тыс.,13,196,6.632653
от 250 до 300 тыс.,87,1321,6.585920
более 500 тыс.,14,222,6.306306


### Вывод

1. Надежность заемщиков с ростом ежемесячного дохода меняется немонотонно. Однако, заемщики с наибольшим (в масштабах выборки) доходом ожидаемо являются наиболее надежными.  
2. Хуже всех платят представители самой широкопредставленной в данных категории (доход от 100 до 200 тыс. рублей): 10695 человек, из которых 9.2% - должники
3. Заемщики с наименьшими и наибольшими (в масштабах выборки) доходами отличаются по надежности всего на 0.09% и входят в первую тройку по надежности (6.22% и 6.31% должников соответственно). 

- Как разные цели кредита влияют на его возврат в срок?  
Сводная таблица по цели кредита и факту наличия задолженности
  - purpose_type - цель взятия кредита
  - debt - количество заемщиков-должников в группе
  - total_clients - общее количество заемщиков в группе
  - debt_percent - процент клиентов с задолженностями в группе

In [46]:
df_grouped_purpose

,debt,total_clients,debt_percent
purpose_type,,,
получение образования,366,3776,9.692797
операции с транспортом,395,4076,9.690873
организация свадьбы,183,2240,8.169643
операции с недвижимостью,773,10237,7.551040


### Вывод

1. Хуже всех с выплатами справляются заемщики, берущие деньги на образование и операции с транспортными средствами (по 9.69% должников)
2. Лучше всех - берущие кредиты на операции с недвижимостью

### Шаг 4. Общий вывод

- В ходе анализа исходного набора данных было проведено __обогащение данных__ (пропуски были устранены в двух колонках с числовыми значениями - 'total_income' и 'days_employed').  
- После __утсранения явных и скрытых дупликатов__ и удаления оставшихся после обогащения пропусков объем датасета сократился на 0.05%
- Были устранены __выбросы__ в колонках 'days_employed' и 'children': в первом случае выбросы возникли в результате системной ошибки (данные были внесены в часах, а не в днях); во втором случае ошибка, вероятнее всего была допущена людьми, вносившими данные в систему
- Во всех колонках, содержащих количественные данные __тип данных__ 'float64' __был изменен__ на 'int64'
- Была проведена __лемматизация__ целей в столбце 'purpose', в результате которой были выделены 4 типа целей взятия кредитов (образование, операции с транспортом, операции с недвижимостью, свадьбы)
- __Данные были категоризированы__ по разным признакам в соответствии с поставленными вопросами
- На все поставленные __вопросы__ были даны __ответы__ (каждый подкреплен соответствующей сводной таблицей)